# Retrieval-Augmented Generation (RAG)
 
**Тема:** Построение, оптимизация и комплексная оценка RAG-систем.

## Введение
В рамках данного задания вам предстоит пройти полный путь ML-инженера при работе с RAG: от сборки базового пайплайна до тонкой настройки ретривера и генератора. Мы будем работать с датасетом **SciFact** (проверка научных фактов).

### Методология Эксперимента
Важно соблюдать гигиену ML-экспериментов. Мы разделили данные на два сета:
1.  **Main Split (200 запросов):** Ваша "Dev" выборка. Все промежуточные прогоны, подбор гиперпараметров (chunk size, top_k) и отладку вы делаете **только** на ней. Мы хотим избежать переобучения под тестовые данные.
2.  **Challenge Split (50 запросов):** Ваша "Test" выборка. Вы используете её **ровно один раз** в самом конце ноутбука для финальной валидации лучшей конфигурации.

---

## 1. Подготовка окружения
Установка зависимостей. Мы используем `LlamaIndex` как оркестратор, `Qdrant` как векторную БД и `HuggingFace` для моделей.

In [18]:
%%capture
!pip install llama-index-core llama-index-llms-huggingface llama-index-embeddings-huggingface llama-index-vector-stores-qdrant llama-index-retrievers-bm25
!pip install qdrant-client ir_datasets ir_measures bitsandbytes accelerate transformers
!pip install pandas matplotlib seaborn tqdm

In [1]:
# JAX Memory Configuration (CRITICAL: must run before imports)
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
print("JAX configured")

JAX configured


In [3]:
import os
import time
import random
import gc
from dataclasses import dataclass, field
from typing import List, Optional, Dict

import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm

# Optimizations for T4 GPU in Colab
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

ARTIFACTS_DIR = os.path.abspath("./artifacts")
QDRANT_PATH = os.path.join(ARTIFACTS_DIR, "qdrant_local")
LOGS_DIR = os.path.join(ARTIFACTS_DIR, "logs")

os.makedirs(QDRANT_PATH, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

## 2. Загрузка данных (SciFact)
Загружаем корпус и запросы. SciFact — это сложный датасет, где запрос часто требует понимания причинно-следственных связей в научных текстах.

In [5]:
import ir_datasets

ds_corpus = ir_datasets.load("beir/scifact")
ds_test = ir_datasets.load("beir/scifact/test")

print("Loading corpus...")
corpus_rows = [{"doc_id": str(d.doc_id), "title": d.title or "", "text": d.text or ""} for d in ds_corpus.docs_iter()]
df_corpus = pd.DataFrame(corpus_rows)

print("Loading queries...")
query_rows = [{"query_id": str(q.query_id), "text": q.text} for q in ds_test.queries_iter()]
df_queries = pd.DataFrame(query_rows)

print("Loading qrels...")
qrel_rows = [{"query_id": str(q.query_id), "corpus_id": str(q.doc_id), "score": int(q.relevance)} for q in ds_test.qrels_iter()]
df_qrels = pd.DataFrame(qrel_rows)

# Map for metrics calculation
qrels_map = df_qrels.groupby("query_id").apply(lambda x: dict(zip(x["corpus_id"], x["score"]))).to_dict()

# Fixed Splits
all_qids = df_queries["query_id"].unique()
rng = np.random.default_rng(42)
rng.shuffle(all_qids)
split_main = all_qids[:200]
split_challenge = all_qids[200:250]

print(f"Corpus size: {len(df_corpus)}")
print(f"Main split: {len(split_main)} queries, Challenge split: {len(split_challenge)} queries")

Loading corpus...
Loading queries...
Loading qrels...
Corpus size: 5183
Main split: 200 queries, Challenge split: 50 queries


C:\Users\Grigorii\AppData\Local\Temp\ipykernel_40136\2705820492.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  qrels_map = df_qrels.groupby("query_id").apply(lambda x: dict(zip(x["corpus_id"], x["score"]))).to_dict()


## 3. Конфигурация и Модели
Мы используем `dataclasses` для строгой типизации конфигов экспериментов.

In [7]:
from llama_index.core import VectorStoreIndex, Document, Settings, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.node_parser import SentenceSplitter, HierarchicalNodeParser
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.retrievers.bm25 import BM25Retriever
from qdrant_client import QdrantClient
from transformers import BitsAndBytesConfig

# --- CONFIG CLASSES ---
@dataclass
class ChunkingConfig:
    chunk_size: int = 512
    chunk_overlap: int = 100
    use_hierarchical: bool = False
    child_chunk_size: int = 128

@dataclass
class RetrievalConfig:
    top_k: int = 5          # Final k documents for LLM
    overfetch_k: int = 15   # Candidates fetched from DB (The "Funnel" top)
    mode: str = "dense"     # "dense" | "hybrid"
    use_reranker: bool = False
    rerank_top_n: int = 5   # Documents to keep after reranking

@dataclass
class LLMConfig:
    model_name: str = "Qwen/Qwen3-4B-Instruct-2507"
    max_new_tokens: int = 256
    context_window: int = 2048
    load_in_4bit: bool = True
    # Subsample for E2E evaluation speed
    e2e_eval_n: int = 10
    prompt_template: str = "Context:\n{context_str}\n\nQuery: {query_str}\nAnswer:"

@dataclass
class EmbeddingConfig:
    model_name: str = "Qwen/Qwen3-Embedding-0.6B"
    truncate_dim: Optional[int] = None # For Matryoshka learning

@dataclass
class RAGConfig:
    name: str = "baseline"
    chunking: ChunkingConfig = field(default_factory=ChunkingConfig)
    retrieval: RetrievalConfig = field(default_factory=RetrievalConfig)
    llm: LLMConfig = field(default_factory=LLMConfig)
    embedding: EmbeddingConfig = field(default_factory=EmbeddingConfig)
    qdrant_collection: str = "scifact_dense_base"
    recreate_collection: bool = False
    rerank_model: str = "Qwen/Qwen3-Reranker-0.6B"

# --- MODEL HELPERS ---
_CACHED_LLM = None
_CACHED_EMBED = None

def unload_embedder():
    global _CACHED_EMBED
    if _CACHED_EMBED:
        del _CACHED_EMBED
        _CACHED_EMBED = None
        Settings.embed_model = None
        gc.collect(); torch.cuda.empty_cache()

def get_embedder(cfg: EmbeddingConfig):
    global _CACHED_EMBED
    if _CACHED_EMBED: return _CACHED_EMBED
    _CACHED_EMBED = HuggingFaceEmbedding(
        model_name=cfg.model_name,
        device="cuda",
        normalize=True,
        truncate_dim=cfg.truncate_dim,
        trust_remote_code=True
    )
    return _CACHED_EMBED

def unload_llm():
    global _CACHED_LLM
    if _CACHED_LLM is not None:
        try:
            del _CACHED_LLM
        except Exception:
            pass
        _CACHED_LLM = None
        gc.collect()
        torch.cuda.empty_cache()
        
def get_llm(cfg: LLMConfig):
    global _CACHED_LLM
    if _CACHED_LLM: return _CACHED_LLM
    from transformers import AutoTokenizer, BitsAndBytesConfig

    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    )

    tok = AutoTokenizer.from_pretrained(cfg.model_name, trust_remote_code=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token

    def completion_to_prompt(prompt: str) -> str:
        messages = [
            {"role": "system", "content": "You are a careful scientific assistant."},
            {"role": "user", "content": prompt},
        ]
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    _CACHED_LLM = HuggingFaceLLM(
        model_name=cfg.model_name,
        context_window=cfg.context_window,
        max_new_tokens=cfg.max_new_tokens,
        model_kwargs={
            "quantization_config": bnb,
            "trust_remote_code": True,
        },
        tokenizer_name=cfg.model_name,
        tokenizer_kwargs={"trust_remote_code": True},
        completion_to_prompt=completion_to_prompt,
        generate_kwargs={
            "do_sample": False,
            "temperature": 0.0,
            "repetition_penalty": 1.05,
            "pad_token_id": tok.pad_token_id,
            "eos_token_id": tok.eos_token_id,
        },
        device_map="auto",
    )
    return _CACHED_LLM


resource module not available on Windows


## 4. Система Метрик (Metrics System)

Для комплексной оценки качества мы используем двухуровневую систему метрик.

### 1. Retrieval Phase (Поиск)
Оценивает, насколько релевантные куски текста (chunks) мы нашли в базе.

**A. С учителем (Supervised / Reference-based):**
Требуют наличия "правильных ответов" (qrels).
*   **`nDCG@k`**: Основная метрика ранжирования. Учитывает не только факт нахождения правильного документа, но и его позицию (чем выше, тем лучше).
*   **`Recall@k`**: Полнота. Какую долю всех релевантных документов мы нашли в топ-k.

**B. Без учителя (Unsupervised / Reference-free):**
Полезны для мониторинга в продакшене, где нет разметки.
*   **`Mean Relevance Score`**: Среднее значение скора (similarity), который выдает ретривер. Показывает "уверенность" модели в найденном.
*   **`Redundancy`**: Избыточность. Показывает, насколько найденные чанки похожи друг на друга. Высокая избыточность — плохо (мы хотим разнообразный контекст).

### 2. Generation Phase (Генерация)
Оценивает качество финального ответа с использованием **LLM-as-a-Judge** (одна LLM оценивает другую).

*   **`Faithfulness` (Answer vs Context):** Верность контексту. Проверяет, что ответ сгенерирован **исключительно** на основе найденных документов, без галлюцинаций.
*   **`Answer Relevancy` (Answer vs Query):** Релевантность вопроса. Проверяет, что сгенерированный текст действительно отвечает на поставленный вопрос пользователя.

In [9]:
import ir_measures
from ir_measures import nDCG, Recall, RR

RUNS_CSV_PATH = os.path.join(LOGS_DIR, "runs.csv")

def compute_redundancy(contexts: List[str]) -> float:
    """Calculates semantic redundancy (higher = more duplicate info)."""
    if not contexts: return 0.0
    unique = len(set(contexts))
    return 1.0 - (unique / len(contexts))

def compute_retrieval_metrics(qrels_map, run_doc_ids, run_contexts, run_scores, k=5):
    # 1. Reference-based Metrics (Ground Truth needed)
    run_ir = {}
    for qid, doc_ids in run_doc_ids.items():
        run_ir[qid] = {doc_id: float(-(rank + 1)) for rank, doc_id in enumerate(doc_ids[:k])}

    measures = [nDCG@k, Recall@k]
    agg = ir_measures.calc_aggregate(measures, qrels_map, run_ir)

    # 2. Reference-free Metrics (No Ground Truth)
    red_vals = [compute_redundancy(ctxs[:k]) for ctxs in run_contexts.values()]
    # Mean Score: How confident is the retriever?
    score_vals = [np.mean(scores[:k]) for scores in run_scores.values() if scores]

    return {
        "ndcg@k": float(agg[nDCG@k]),
        "recall@k": float(agg[Recall@k]),
        "ref_free_mean_score": float(np.mean(score_vals)) if score_vals else 0.0,
        "ref_free_redundancy": float(np.mean(red_vals)) if red_vals else 0.0
    }

def log_run(cfg, split, stage, run_name, metrics):
    row = {
        "ts": time.strftime("%H:%M:%S", time.gmtime()),
        "run_name": run_name,
        "split": split,
        "stage": stage,
        **metrics
    }
    df_old = pd.read_csv(RUNS_CSV_PATH) if os.path.exists(RUNS_CSV_PATH) else pd.DataFrame()
    pd.concat([df_old, pd.DataFrame([row])], ignore_index=True).to_csv(RUNS_CSV_PATH, index=False)

def show_leaderboard(stage="retrieval", split="main"):
    if not os.path.exists(RUNS_CSV_PATH): return
    df = pd.read_csv(RUNS_CSV_PATH)
    df_sub = df[(df["stage"]==stage) & (df["split"]==split)].copy()
    if df_sub.empty: return

    # Sort by key metric
    sort_col = "ndcg@k" if stage == "retrieval" else "faithfulness"
    if sort_col in df_sub.columns:
        df_sub = df_sub.sort_values(sort_col, ascending=False)

    # Select columns to display
    cols_ret = ["run_name", "ndcg@k", "recall@k", "ref_free_mean_score", "ref_free_redundancy", "latency_s"]
    cols_gen = ["run_name", "faithfulness", "answer_relevancy", "latency_s"]

    cols = cols_ret if stage == "retrieval" else cols_gen
    print(f"\n🏆 LEADERBOARD [{stage.upper()} | {split.upper()}] 🏆")
    display(df_sub[[c for c in cols if c in df_sub.columns]])

def _get_df(split): return df_queries[df_queries["query_id"].isin(split_main if split=="main" else split_challenge)]

def run_retrieval(cfg: RAGConfig, split: str, run_name: str):
    unload_embedder()
    index, retriever, _ = build_rag_pipeline(cfg, df_corpus, load_llm=False)

    df_q = _get_df(split)
    results_dids, results_txts, results_scores = {}, {}, {}

    t0 = time.time()
    for row in tqdm(df_q.itertuples(index=False), total=len(df_q), desc=f"Retr {run_name}"):
        nodes = retriever.retrieve(row.text)
        # De-duplicate by doc_id to avoid metrics skew
        seen, dids, txts, scores = set(), [], [], []
        for n in nodes:
            if n.metadata["doc_id"] not in seen:
                seen.add(n.metadata["doc_id"])
                dids.append(n.metadata["doc_id"])
                txts.append(n.get_text())
                scores.append(n.score if n.score else 0.0)
        results_dids[str(row.query_id)] = dids
        results_txts[str(row.query_id)] = txts
        results_scores[str(row.query_id)] = scores

    # FIX: filter qrels_map based on current split
    split_qids = set(df_q["query_id"].astype(str))
    qrels_map_filtered = {qid: docs for qid, docs in qrels_map.items() if qid in split_qids}
    
    latency = time.time() - t0

    metrics = compute_retrieval_metrics(qrels_map_filtered, results_dids, results_txts, results_scores, k=cfg.retrieval.top_k)
    metrics["latency_s"] = latency
    log_run(cfg, split, "retrieval", run_name, metrics)
    show_leaderboard("retrieval", split)
    unload_embedder()

def run_generation(cfg: RAGConfig, split: str, run_name: str):
    """Generates answers and saves them to JSON for subsequent evaluation."""
    # 1. Retrieval Phase
    unload_embedder()
    index, retriever, _ = build_rag_pipeline(cfg, df_corpus, load_llm=False)

    df_q = _get_df(split).head(cfg.llm.e2e_eval_n)
    records = []
    for row in tqdm(df_q.itertuples(index=False), total=len(df_q), desc="Retrieving Contexts"):
        nodes = retriever.retrieve(row.text)
        ctx_list = [n.get_text() for n in nodes[:cfg.retrieval.top_k]]
        records.append({"query_id": str(row.query_id), "q": row.text, "ctx": "\n".join(ctx_list)})

    unload_embedder()

    # 2. Generation Phase
    Settings.embed_model = None
    llm = get_llm(cfg.llm)

    t0 = time.time()
    for item in tqdm(records, desc="Generating"):
        prompt = cfg.llm.prompt_template.format(context_str=item["ctx"], query_str=item["q"])
        try: 
            item["a"] = llm.complete(prompt).text
        except: 
            item["a"] = ""
    latency = time.time() - t0

    # 3. Save results
    output_path = os.path.join(LOGS_DIR, f"generations_{run_name}_{split}.json")
    import json
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump({
            "records": records, 
            "latency_s": latency, 
            "cfg_name": cfg.name,
            "run_name": run_name,
            "split": split
        }, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Generations saved to {output_path}")
    return output_path


def run_evaluation(generation_file: str, eval_name: Optional[str] = None):
    """Evaluates already generated answers."""
    import json
    
    # Load generated data
    with open(generation_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    records = data["records"]
    latency = data["latency_s"]
    split = data.get("split", "main")
    run_name = eval_name if eval_name else data.get("run_name", "eval")
    
    # Load LLM for Judge
    Settings.embed_model = None
    llm = get_llm(LLMConfig())
    
    # Judge Phase
    faith_scores = []
    rel_scores = []

    for item in tqdm(records, desc=f"Judging {run_name}"):
        if not item.get("a", ""):
            faith_scores.append(0.0); rel_scores.append(0.0)
            continue

        # Metric 1: Faithfulness (Context vs Answer)
        p_faith = f"Context: {item['ctx'][:1000]}\nAnswer: {item['a']}\nDoes the Answer use the Context? YES/NO."
        res_f = llm.complete(p_faith, max_new_tokens=5).text.upper()
        faith_scores.append(1.0 if "YES" in res_f else 0.0)

        # Metric 2: Answer Relevancy (Question vs Answer)
        p_rel = f"Question: {item['q']}\nAnswer: {item['a']}\nDoes the Answer address the Question? YES/NO."
        res_r = llm.complete(p_rel, max_new_tokens=5).text.upper()
        rel_scores.append(1.0 if "YES" in res_r else 0.0)

    metrics = {
        "faithfulness": np.mean(faith_scores),
        "answer_relevancy": np.mean(rel_scores),
        "latency_s": latency
    }
    
    log_run(RAGConfig(name=run_name), split, "e2e", run_name, metrics)
    show_leaderboard("e2e", split)
    return metrics


def run_e2e(cfg: RAGConfig, split: str, run_name: str, generation_file: Optional[str] = None):
    """
    End-to-end evaluation with option to reuse generations.
    
    Args:
        cfg: RAG configuration
        split: "main" or "challenge"
        run_name: Experiment name
        generation_file: Path to previously generated answers file (optional)
    """
    if generation_file and os.path.exists(generation_file):
        # Reuse existing generation
        print(f"📂 Loading generations from {generation_file}")
        return run_evaluation(generation_file, run_name)
    else:
        # Full cycle: generation + evaluation
        gen_file = run_generation(cfg, split, run_name)
        return run_evaluation(gen_file, run_name)

## Task 0: Реализация Baseline Pipeline

Ваша первая задача — реализовать функцию `build_rag_pipeline`. На данном этапе требуется собрать простую архитектуру Dense Retrieval.

**Требования к реализации:**
1.  Инициализация моделей через `Settings` (используйте `get_embedder` и `get_llm`).
2.  Подключение к `QdrantVectorStore`.
3.  Логика индексации: Если коллекция не существует (или `recreate_collection=True`), создать её из `df_corpus`, используя `SentenceSplitter`. Если существует — загрузить существующий индекс.
4.  Возврат объектов `index`, `retriever` и `query_engine`.

Обратите внимание на параметр `overfetch_k` в конфиге. Ретривер должен извлекать именно это количество кандидатов.

In [11]:
from typing import Any, Tuple
from dataclasses import replace

from llama_index.core.schema import NodeWithScore

In [13]:
# Helper
def _collection_exists(client: QdrantClient, name: str) -> bool:
    try:
        return bool(client.collection_exists(name))  # newer qdrant-client
    except Exception:
        try:
            cols = client.get_collections()
            return any(c.name == name for c in cols.collections)
        except Exception:
            return False

In [15]:
# Hybrid RRF retriever (robust, no dependency on QueryFusionRetriever API)
class HybridRRFRetriever:
    """
    Simple Reciprocal Rank Fusion over two retrievers.
    Returns top_k NodeWithScore with fused score.
    """
    def __init__(self, dense_retriever, bm25_retriever, top_k: int = 20, rrf_k: int = 60):
        self.dense = dense_retriever
        self.bm25 = bm25_retriever
        self.top_k = top_k
        self.rrf_k = rrf_k

    def retrieve(self, query: str):
        dense_nodes = self.dense.retrieve(query) or []
        bm25_nodes = self.bm25.retrieve(query) or []

        # Key for fusion: prefer (doc_id + text hash) to avoid collisions
        def _key(n: Any) -> str:
            md = getattr(n, "metadata", {}) or {}
            doc_id = str(md.get("doc_id", ""))
            txt = ""
            try:
                txt = n.get_text()
            except Exception:
                try:
                    txt = n.node.get_text()
                except Exception:
                    txt = ""
            return f"{doc_id}::{hash(txt)}"

        fused = {}  # key -> (best_node, score)

        def _add(list_nodes, weight=1.0):
            for rank, n in enumerate(list_nodes):
                k = _key(n)
                score = weight * (1.0 / (self.rrf_k + rank + 1))
                if k not in fused:
                    fused[k] = [n, 0.0]
                fused[k][1] += score

        _add(dense_nodes)
        _add(bm25_nodes)

        out = []
        for n, s in fused.values():
            # set/update score if possible
            try:
                n.score = float(s)
                out.append(n)
            except Exception:
                # ensure NodeWithScore shape if needed
                try:
                    out.append(NodeWithScore(node=n.node, score=float(s)))
                except Exception:
                    out.append(n)

        out.sort(key=lambda x: float(getattr(x, "score", 0.0) or 0.0), reverse=True)
        return out[: self.top_k]

In [16]:
_CACHED_RERANK = None

def unload_reranker():
    global _CACHED_RERANK
    if _CACHED_RERANK is not None:
        tok, mdl, *_ = _CACHED_RERANK
        del tok, mdl
        _CACHED_RERANK = None
        import gc
        gc.collect()
        torch.cuda.empty_cache()

def _format_instruction(instruction: str | None, query: str, doc: str) -> str:
    if instruction is None:
        # можно чуть "заточить" под SciFact
        instruction = "Given a scientific claim, retrieve passages that support or refute it."
    return f"<Instruct>: {instruction}\n<Query>: {query}\n<Document>: {doc}"

def get_reranker(model_name: str, max_reranker_length: int = 2048):
    """
    Qwen3 reranker is a generative reranker:
    score = P(yes | formatted_input).
    """
    global _CACHED_RERANK
    if _CACHED_RERANK is not None:
        return _CACHED_RERANK
    
    import torch
    import torch.nn.functional as F
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    )

    tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_side="left")
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token

    mdl = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        quantization_config=bnb,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
    ).eval()

    token_false_id = tok.convert_tokens_to_ids("no")
    token_true_id  = tok.convert_tokens_to_ids("yes")

    # этот шаблон (prefix/suffix) — из примеров Qwen3 reranker
    prefix = (
        '<|im_start|>system\n'
        ' Judge whether the Document meets the requirements based on the Query and the Instruct provided. '
        'Note that the answer can only be "yes" or "no".<|im_end|>\n'
        '<|im_start|>user\n'
    )
    suffix = '<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n'

    prefix_tokens = tok.encode(prefix, add_special_tokens=False)
    suffix_tokens = tok.encode(suffix, add_special_tokens=False)

    _CACHED_RERANK = (tok, mdl, token_true_id, token_false_id, prefix_tokens, suffix_tokens, max_reranker_length)
    return _CACHED_RERANK

@torch.no_grad()
def qwen_rerank_scores(query: str, docs: list[str], model_name: str, batch_size: int = 8, instruction: str | None = None):
    import torch.nn.functional as F
    
    tok, mdl, true_id, false_id, prefix_toks, suffix_toks, max_len = get_reranker(model_name)

    pairs = [_format_instruction(instruction, query, d) for d in docs]

    scores = []
    for i in range(0, len(pairs), batch_size):
        batch_pairs = pairs[i:i+batch_size]

        inputs = tok(
            batch_pairs,
            padding=False,
            truncation="longest_first",
            return_attention_mask=False,
            max_length=max_len - len(prefix_toks) - len(suffix_toks),
        )

        # добавляем prefix/suffix
        for j, ids in enumerate(inputs["input_ids"]):
            inputs["input_ids"][j] = prefix_toks + ids + suffix_toks

        inputs = tok.pad(inputs, padding="max_length", return_tensors="pt", max_length=max_len)

        # перенос на устройство модели
        inputs = {k: v.to(mdl.device) for k, v in inputs.items()}

        logits = mdl(**inputs).logits[:, -1, :]          # (B, vocab)
        yes = logits[:, true_id]
        no  = logits[:, false_id]
        two = torch.stack([no, yes], dim=1)              # (B,2)
        p_yes = F.log_softmax(two, dim=1)[:, 1].exp()    # вероятность "yes"
        scores.extend(p_yes.detach().float().cpu().tolist())

    return scores

In [17]:
class RerankRetriever:
    def __init__(self, base_retriever, model_name: str, top_n: int, batch_size: int = 8):
        self.base = base_retriever
        self.model_name = model_name
        self.top_n = top_n
        self.batch_size = batch_size

    @torch.no_grad()
    def retrieve(self, query: str):
        nodes = self.base.retrieve(query)
        if not nodes:
            return nodes

        texts = [n.get_text() for n in nodes]
        scores = qwen_rerank_scores(
            query=query,
            docs=texts,
            model_name=self.model_name,
            batch_size=self.batch_size,
            instruction="Given a scientific claim, retrieve passages that support or refute it."
        )

        ranked = sorted(zip(nodes, scores), key=lambda x: x[1], reverse=True)
        out = []
        for n, s in ranked[: self.top_n]:
            n.score = float(s)
            out.append(n)
        return out

In [19]:
from qdrant_client.http import models as qmodels
from qdrant_client import QdrantClient

_QDRANT_CLIENT = None

def get_qdrant_client():
    global _QDRANT_CLIENT
    if _QDRANT_CLIENT is None:
        _QDRANT_CLIENT = QdrantClient(path=QDRANT_PATH)
    return _QDRANT_CLIENT

def close_qdrant_client():
    global _QDRANT_CLIENT
    if _QDRANT_CLIENT is not None:
        try:
            _QDRANT_CLIENT.close()
        except Exception:
            pass
        _QDRANT_CLIENT = None

In [20]:
def build_rag_pipeline(cfg: RAGConfig, df_corpus: pd.DataFrame, load_llm: bool = True):
    # 0. Ensure paths exist
    os.makedirs(QDRANT_PATH, exist_ok=True)
    # 1. Setup Models
    Settings.embed_model = get_embedder(cfg.embedding)
    Settings.llm = get_llm(cfg.llm) if load_llm else None
    
    # 2. Vector Store Setup 
    client = get_qdrant_client()

    dim = len(Settings.embed_model.get_text_embedding("dim_probe"))

    # recreate / create collection explicitly
    if cfg.recreate_collection:
        client.recreate_collection(
            collection_name=cfg.qdrant_collection,
            vectors_config=qmodels.VectorParams(size=dim, distance=qmodels.Distance.COSINE),
        )
        must_reindex = True
    else:
        # иначе пытаемся получить; если нет — создаём
        try:
            client.get_collection(cfg.qdrant_collection)
            must_reindex = False
        except Exception:
            client.create_collection(
                collection_name=cfg.qdrant_collection,
                vectors_config=qmodels.VectorParams(size=dim, distance=qmodels.Distance.COSINE),
            )
            must_reindex = True

    vector_store = QdrantVectorStore(client=client, collection_name=cfg.qdrant_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # 3. Indexing Logic 
    # Check if collection exists.
    # If yes & not recreate -> load index.
    # Else -> create documents, setup splitter, build index.
    if (not must_reindex) and (not cfg.chunking.use_hierarchical):
        # быстрый путь: коллекция уже есть, просто подключаем индекс к vector_store
        index = VectorStoreIndex.from_vector_store(vector_store=vector_store, storage_context=storage_context)
    else:
        # создаём документы
        documents = []
        for r in df_corpus.itertuples(index=False):
            full_text = (str(r.title).strip() + "\n\n" + str(r.text).strip()).strip()
            documents.append(
                Document(
                    text=full_text,
                    metadata={"doc_id": str(r.doc_id), "title": str(getattr(r, "title", "") or "")},
                )
            )

        if cfg.chunking.use_hierarchical:
            # Hierarchical: индексируем leaf, а по возможности возвращаем parent через AutoMergingRetriever
            parser = HierarchicalNodeParser.from_defaults(
                chunk_sizes=[cfg.chunking.chunk_size, cfg.chunking.child_chunk_size],
                chunk_overlap=cfg.chunking.chunk_overlap,
            )
            all_nodes = parser.get_nodes_from_documents(documents)

            leaf_nodes = []
            for n in all_nodes:
                child = getattr(n, "child_nodes", None)
                if not child:
                    leaf_nodes.append(n)

            # сохраняем все ноды в docstore, чтобы AutoMerging смог склеивать
            try:
                storage_context.docstore.add_documents(all_nodes)
            except Exception:
                pass

            index = VectorStoreIndex(nodes=leaf_nodes, storage_context=storage_context, show_progress=True)
        else:
            splitter = SentenceSplitter(
                chunk_size=cfg.chunking.chunk_size,
                chunk_overlap=cfg.chunking.chunk_overlap,
            )
            index = VectorStoreIndex.from_documents(
                documents,
                storage_context=storage_context,
                transformations=[splitter],
                show_progress=True,
            )

    # 4. Retrieval Construction (TODO)
    # retriever = ... (use cfg.retrieval.overfetch_k)
    # query_engine = ...
    dense_retriever = index.as_retriever(similarity_top_k=cfg.retrieval.overfetch_k)

    # Optional: hierarchical “return parents”
    retriever = dense_retriever
    if cfg.chunking.use_hierarchical:
        try:
            from llama_index.core.retrievers import AutoMergingRetriever
            retriever = AutoMergingRetriever(dense_retriever, storage_context, verbose=False)
        except Exception:
            # fallback: just dense leaf chunks
            retriever = dense_retriever

    # Hybrid mode (dense + BM25) with RRF
    if cfg.retrieval.mode == "hybrid":
        # Build BM25 over the SAME chunking strategy (best-effort).
        # NOTE: BM25 doesn't depend on qdrant; we can create it on the fly.
        if not cfg.chunking.use_hierarchical:
            splitter = SentenceSplitter(
                chunk_size=cfg.chunking.chunk_size,
                chunk_overlap=cfg.chunking.chunk_overlap,
            )
            docs_for_bm25 = []
            for r in df_corpus.itertuples(index=False):
                full_text = (str(r.title).strip() + "\n\n" + str(r.text).strip()).strip()
                docs_for_bm25.append(
                    Document(text=full_text, metadata={"doc_id": str(r.doc_id), "title": str(r.title or "")})
                )
            bm25_nodes = splitter.get_nodes_from_documents(docs_for_bm25)
        else:
            # For hierarchical: use leaf nodes as bm25 units
            # (parents are reconstructed by AutoMerging on dense path; bm25 stays leaf-level)
            try:
                # if we indexed from scratch, docstore may contain leaf nodes; otherwise rebuild quickly
                bm25_nodes = []
                # safest rebuild:
                parser = HierarchicalNodeParser.from_defaults(
                    chunk_sizes=[cfg.chunking.chunk_size, cfg.chunking.child_chunk_size],
                    chunk_overlap=cfg.chunking.chunk_overlap,
                )
                docs_for_bm25 = []
                for r in df_corpus.itertuples(index=False):
                    full_text = (str(r.title).strip() + "\n\n" + str(r.text).strip()).strip()
                    docs_for_bm25.append(Document(text=full_text, metadata={"doc_id": str(r.doc_id), "title": str(r.title or "")}))
                all_nodes = parser.get_nodes_from_documents(docs_for_bm25)
                for n in all_nodes:
                    child = getattr(n, "child_nodes", None)
                    if not child:
                        bm25_nodes.append(n)
            except Exception:
                bm25_nodes = []

        bm25_retriever = BM25Retriever.from_defaults(
            nodes=bm25_nodes,
            similarity_top_k=cfg.retrieval.overfetch_k,
        )

        retriever = HybridRRFRetriever(
            dense_retriever=dense_retriever,
            bm25_retriever=bm25_retriever,
            top_k=cfg.retrieval.overfetch_k,
            rrf_k=60,
        )

    # Optional reranking (funnel: overfetch_k -> rerank_top_n)
    if cfg.retrieval.use_reranker:
        retriever = RerankRetriever(
            base_retriever=retriever,
            model_name=cfg.rerank_model,
            top_n=max(cfg.retrieval.rerank_top_n, cfg.retrieval.top_k),
            batch_size=8
        )

    # Query engine (not strictly needed for твоих run_* функций, но возвращаем как требуется)
    query_engine = index.as_query_engine(retriever=retriever) if load_llm else None

    # return index, retriever, query_engine
    return index, retriever, query_engine

In [31]:
# TEST YOUR BASELINE
baseline_cfg = RAGConfig(
    name="baseline",
    chunking=ChunkingConfig(chunk_size=512, chunk_overlap=50),
    retrieval=RetrievalConfig(top_k=5, overfetch_k=15),
    qdrant_collection="scifact_base_512",
    recreate_collection=True
)

print("Running Baseline...")
# 1. Run Retrieval
# TODO: Uncomment after implementing pipeline
run_retrieval(baseline_cfg, "main", "baseline")

# 2. Check Results
show_leaderboard("retrieval", "main")

Running Baseline...


D:\Soft\Anaconda\Lib\site-packages\torch\nn\modules\module.py:1326: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


LLM is explicitly disabled. Using MockLLM.


C:\Users\Grigorii\AppData\Local\Temp\ipykernel_34096\608474731.py:15: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Parsing nodes:   0%|          | 0/5183 [00:00<?, ?it/s]

...

Retr baseline:   0%|          | 0/200 [00:00<?, ?it/s]


🏆 LEADERBOARD [RETRIEVAL | MAIN] 🏆


,run_name,ndcg@k,recall@k,ref_free_mean_score,ref_free_redundancy,latency_s
0,baseline,0.674031,0.7695,0.573575,0.0,22.45268


Embeddings have been explicitly disabled. Using MockEmbedding.

🏆 LEADERBOARD [RETRIEVAL | MAIN] 🏆


,run_name,ndcg@k,recall@k,ref_free_mean_score,ref_free_redundancy,latency_s
0,baseline,0.674031,0.7695,0.573575,0.0,22.45268


## Task 1: Chunking Strategies

Размер контекста критически влияет на качество поиска.

**Задание:**
1.  **Small vs Large:** Создайте конфигурации для `chunk_size=256` и `chunk_size=1024`. Запустите эксперименты. *Напоминание: при изменении чанкинга необходимо менять имя коллекции или ставить `recreate_collection=True`.*
2.  **Hierarchical Chunking:** Реализуйте иерархический чанкинг (`use_hierarchical=True`). Для этого в `build_rag_pipeline` необходимо добавить проверку конфига и использование `HierarchicalNodeParser`. Этот метод индексирует мелкие чанки, но возвращает контекст родительских (крупных) блоков.

In [20]:
# TODO: Update build_rag_pipeline to support HierarchicalNodeParser

# TODO: Define Configs
# cfg_256 = ...
# cfg_1024 = ...
# cfg_hier = ...
cfg_256 = RAGConfig(
    name="chunk_256",
    chunking=ChunkingConfig(chunk_size=256, chunk_overlap=50),
    retrieval=RetrievalConfig(top_k=5, overfetch_k=20, mode="dense"),
    qdrant_collection="scifact_chunk_256",
    recreate_collection=True
)

cfg_1024 = RAGConfig(
    name="chunk_1024",
    chunking=ChunkingConfig(chunk_size=1024, chunk_overlap=100),
    retrieval=RetrievalConfig(top_k=5, overfetch_k=20, mode="dense"),
    qdrant_collection="scifact_chunk_1024",
    recreate_collection=True
)

cfg_hier = RAGConfig(
    name="hierarchical",
    chunking=ChunkingConfig(
        chunk_size=1024,         # parent
        child_chunk_size=256,    # leaf indexed
        chunk_overlap=80,
        use_hierarchical=True
    ),
    retrieval=RetrievalConfig(top_k=5, overfetch_k=25, mode="dense"),
    qdrant_collection="scifact_hier_1024_256",
    recreate_collection=True,  # рекомендовано (нужны отношения parent/child)
)
# TODO: Run Experiments
# run_retrieval(cfg_256, "main", "chunk_256")
# show_leaderboard("retrieval", "main")
run_retrieval(cfg_256, "main", "chunk_256")
run_retrieval(cfg_1024, "main", "chunk_1024")
run_retrieval(cfg_hier, "main", "hierarchical")

show_leaderboard("retrieval", "main")

D:\Soft\Anaconda\Lib\site-packages\torch\nn\modules\module.py:1326: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


LLM is explicitly disabled. Using MockLLM.


C:\Users\Grigorii\AppData\Local\Temp\ipykernel_40064\3262185644.py:15: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Parsing nodes:   0%|          | 0/5183 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Retr hierarchical:   0%|          | 0/200 [00:00<?, ?it/s]


🏆 LEADERBOARD [RETRIEVAL | MAIN] 🏆


,run_name,ndcg@k,recall@k,ref_free_mean_score,ref_free_redundancy,latency_s
3,hierarchical,0.683837,0.76700,0.565709,0.0,26.681055
1,chunk_256,0.678425,0.76075,0.578076,0.0,26.496207
2,chunk_1024,0.674990,0.76325,0.572649,0.0,21.664925
0,baseline,0.674031,0.76950,0.573575,0.0,22.452680


Embeddings have been explicitly disabled. Using MockEmbedding.

🏆 LEADERBOARD [RETRIEVAL | MAIN] 🏆


,run_name,ndcg@k,recall@k,ref_free_mean_score,ref_free_redundancy,latency_s
3,hierarchical,0.683837,0.76700,0.565709,0.0,26.681055
1,chunk_256,0.678425,0.76075,0.578076,0.0,26.496207
2,chunk_1024,0.674990,0.76325,0.572649,0.0,21.664925
0,baseline,0.674031,0.76950,0.573575,0.0,22.452680


## Task 2: Optimization & Advanced Retrieval

В этом блоке мы реализуем продвинутые техники для улучшения качества и эффективности.

### 2.1 Matryoshka Embeddings (Optimization)
Модель `Qwen/Qwen3-Embedding` поддерживает **Matryoshka Representation Learning (MRL)**. Это позволяет усекать размерность векторов (например, с 1024 до 512) с минимальной потерей качества, что экономит память и ускоряет поиск.
**Задание:** Измените `truncate_dim` в `EmbeddingConfig` (например, на 512) и проверьте влияние на метрики.

### 2.2 Advanced Retrieval (Hybrid + Rerank)
Вам необходимо модифицировать `build_rag_pipeline` (или создать `build_advanced_pipeline`), добавив следующую логику:
1.  **Hybrid Search:** Если `cfg.retrieval.mode == 'hybrid'`, создайте `BM25Retriever` и объедините его с векторным через `QueryFusionRetriever` (алгоритм Reciprocal Rank Fusion).
2.  **Reranking:** Если `cfg.retrieval.use_reranker == True`, добавьте `SentenceTransformerRerank` в список `node_postprocessors` для `QueryEngine`. Реренкер должен принимать на вход топ-K кандидатов из этапа 1 (`overfetch_k`) и оставлять лучшие N (`rerank_top_n`).

**Concept:** Funnel Architecture (Broad Search -> Narrow Filtering).

In [30]:
# TODO: Upgrade Pipeline Logic (Hybrid, Reranker)
# def build_rag_pipeline(...):
#     ...

# TODO: Run Experiments
# cfg_matryoshka = ...
# cfg_hybrid_rerank = ...
# run_retrieval(cfg_hybrid_rerank, "main", "advanced_full")
# show_leaderboard("retrieval", "main")
cfg_matryoshka = RAGConfig(
    name="matryoshka_512",
    chunking=ChunkingConfig(chunk_size=512, chunk_overlap=50),
    retrieval=RetrievalConfig(top_k=5, overfetch_k=20, mode="dense"),
    embedding=EmbeddingConfig(model_name="Qwen/Qwen3-Embedding-0.6B", truncate_dim=512),
    qdrant_collection="scifact_mrl512_chunk512",
    recreate_collection=True,  # меняется эмбеддинг-геометрия => пересоздать
)

run_retrieval(cfg_matryoshka, "main", "matryoshka_512")
show_leaderboard("retrieval", "main")

LLM is explicitly disabled. Using MockLLM.


C:\Users\Grigorii\AppData\Local\Temp\ipykernel_40064\3262185644.py:15: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Parsing nodes:   0%|          | 0/5183 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Retr matryoshka_512:   0%|          | 0/200 [00:00<?, ?it/s]


🏆 LEADERBOARD [RETRIEVAL | MAIN] 🏆


,run_name,ndcg@k,recall@k,ref_free_mean_score,ref_free_redundancy,latency_s
3,hierarchical,0.683837,0.767000,0.565709,0.0,26.681055
1,chunk_256,0.678425,0.760750,0.578076,0.0,26.496207
2,chunk_1024,0.674990,0.763250,0.572649,0.0,21.664925
0,baseline,0.674031,0.769500,0.573575,0.0,22.452680
4,matryoshka_512,0.655511,0.751583,0.605584,0.0,19.032776


Embeddings have been explicitly disabled. Using MockEmbedding.

🏆 LEADERBOARD [RETRIEVAL | MAIN] 🏆


,run_name,ndcg@k,recall@k,ref_free_mean_score,ref_free_redundancy,latency_s
3,hierarchical,0.683837,0.767000,0.565709,0.0,26.681055
1,chunk_256,0.678425,0.760750,0.578076,0.0,26.496207
2,chunk_1024,0.674990,0.763250,0.572649,0.0,21.664925
0,baseline,0.674031,0.769500,0.573575,0.0,22.452680
4,matryoshka_512,0.655511,0.751583,0.605584,0.0,19.032776


In [24]:
cfg_hybrid_rerank = RAGConfig(
    name="hybrid_rerank",
    chunking=ChunkingConfig(chunk_size=512, chunk_overlap=50),
    retrieval=RetrievalConfig(
        top_k=5,
        overfetch_k=40,      # шире воронка
        mode="hybrid",
        use_reranker=True,
        rerank_top_n=8       # после реранка оставляем N
    ),
    qdrant_collection="scifact_hybrid_512",
    recreate_collection=True,
    rerank_model="Qwen/Qwen3-Reranker-0.6B",
)

In [ ]:
run_retrieval(cfg_hybrid_rerank, "main", "advanced_full")
show_leaderboard("retrieval", "main")

## Task 3: Generator Tuning (Prompt Engineering)

Качество ретривала (nDCG) мы оптимизировали. Теперь фокус на Генераторе.

**Задание:**
Модифицируйте `prompt_template` в `LLMConfig`. Цель — максимизировать метрики **Faithfulness** и **Answer Relevancy**.
Рекомендуемые техники:
*   **Role Prompting:** ("You are an expert scientist...")
*   **Constraint Enforcement:** ("Answer ONLY based on the context. If unsure, say 'I don't know'.")
*   **Chain-of-Thought:** ("Let's analyze the context step by step...")

In [26]:
# TODO: Create Prompt Config
# cfg_prompt = ...
# run_e2e(cfg_prompt, "main", "prompt_tuned")
# show_leaderboard("e2e", "main")

prompt_tuned = """You are a careful scientific assistant.

RULES:
- Answer using ONLY the provided context.
- If the context does not contain enough information to answer, say: "I don't know based on the provided context."
- Be concise and directly address the query.
- When possible, quote or paraphrase the exact supporting sentence(s) from the context.

Context:
{context_str}

Query: {query_str}

Answer (follow the RULES):"""

cfg_prompt = replace(
    cfg_hybrid_rerank,
    name="prompt_tuned",
    llm=LLMConfig(
        model_name=cfg_hybrid_rerank.llm.model_name,
        max_new_tokens=256,
        context_window=2048,
        load_in_4bit=True,
        e2e_eval_n=10,
        prompt_template=prompt_tuned,
    ),
)

In [28]:
llm = get_llm(cfg_prompt.llm)
print(llm.complete("Say 'YES' and nothing else.").text)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

D:\Soft\Anaconda\Lib\site-packages\accelerate\utils\modeling.py:804: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


YES


In [30]:
run_e2e(cfg_prompt, "main", "prompt_tuned")
show_leaderboard("e2e", "main")

LLM is explicitly disabled. Using MockLLM.


C:\Users\Grigorii\AppData\Local\Temp\ipykernel_40136\2074746868.py:8: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Collection <scifact_hybrid_512> contains 57460 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  _QDRANT_CLIENT = QdrantClient(path=QDRANT_PATH)
C:\Users\Grigorii\AppData\Local\Temp\ipykernel_40136\2042256602.py:15: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Parsing nodes:   0%|          | 0/5183 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

D:\Soft\Anaconda\Lib\site-packages\qdrant_client\qdrant_client.py:1868: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 59508 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  return self._client.upload_points(


...

Judging prompt_tuned:   0%|          | 0/10 [00:00<?, ?it/s]


🏆 LEADERBOARD [E2E | MAIN] 🏆


,run_name,faithfulness,answer_relevancy,latency_s
9,prompt_tuned,0.7,0.7,111.162622
6,prompt_tuned,0.0,0.0,341.332121
8,prompt_tuned,0.0,0.0,310.618638



🏆 LEADERBOARD [E2E | MAIN] 🏆


,run_name,faithfulness,answer_relevancy,latency_s
9,prompt_tuned,0.7,0.7,111.162622
6,prompt_tuned,0.0,0.0,341.332121
8,prompt_tuned,0.0,0.0,310.618638


## 4. Final Challenge

Выберите одну лучшую конфигурацию по совокупности метрик. Запустите её на отложенной выборке **Challenge Split**.
Сравните результаты с Baseline на этом же сплите.

In [37]:
baseline_cfg = RAGConfig(
    name="baseline",
    chunking=ChunkingConfig(chunk_size=512, chunk_overlap=50),
    retrieval=RetrievalConfig(top_k=5, overfetch_k=15),
    qdrant_collection="scifact_base_512",
    recreate_collection=True
)

In [39]:
# TODO: Final Evaluation
# run_retrieval(best_cfg, "challenge", "final_submission")
# run_e2e(best_cfg, "challenge", "final_submission")

run_retrieval(baseline_cfg, "challenge", "baseline_submission")
run_e2e(baseline_cfg, "challenge", "baseline_submission")

print("FINAL RESULTS:")
show_leaderboard("retrieval", "challenge")
show_leaderboard("e2e", "challenge")

LLM is explicitly disabled. Using MockLLM.


C:\Users\Grigorii\AppData\Local\Temp\ipykernel_40136\2042256602.py:15: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Parsing nodes:   0%|          | 0/5183 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

,run_name,faithfulness,answer_relevancy,latency_s
11,final_submission,0.9,1.0,332.567388
13,baseline_submission,0.9,1.0,332.301167


FINAL RESULTS:

🏆 LEADERBOARD [RETRIEVAL | CHALLENGE] 🏆


,run_name,ndcg@k,recall@k,ref_free_mean_score,ref_free_redundancy,latency_s
10,final_submission,0.762624,0.850000,0.510316,0.0,490.441892
12,baseline_submission,0.636677,0.763333,0.556162,0.0,7.838024



🏆 LEADERBOARD [E2E | CHALLENGE] 🏆


,run_name,faithfulness,answer_relevancy,latency_s
11,final_submission,0.9,1.0,332.567388
13,baseline_submission,0.9,1.0,332.301167
